In [76]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from utils import read_plux_data,read_tobii_data
from tqdm import tqdm as loop

In [2]:
data_root = "./data/exp1"

In [49]:
def get_files(folder,fps_video=25,fps_gaze=50):
    label_file = [os.path.join(folder,i) for i in os.listdir(folder) if ".txt" in i][0]
    tobii_folder = [os.path.join(folder,i) for i in os.listdir(folder) if os.path.isdir(os.path.join(folder,i))][0]
    label_df = read_plux_data(label_file)
    gaze_df = read_tobii_data(os.path.join(tobii_folder,"gazedata.gz"))
    frames = [int(i/(fps_gaze/fps_video)) for i in range(len(gaze_df))]
    gaze_df["Frame"] = frames
    screen_video = os.path.join(tobii_folder,"scenevideo.mp4")
    return screen_video,gaze_df

In [50]:
screen_video,gaze_df = get_files(data_root)

In [52]:
gaze_df.head()

,TIME,pLeft,pRight,gazeX,gazeY,Frame
0,1.651824e+09,3.176541,3.380422,0.530833,0.559420,0
1,1.651824e+09,3.179094,3.383096,0.530842,0.559625,0
2,1.651824e+09,3.184719,3.381771,0.530912,0.559550,1
3,1.651824e+09,3.190416,3.380764,0.530796,0.559932,1
4,1.651824e+09,3.197776,3.385649,0.530754,0.559935,2


In [97]:
def generate_fucosed_region(screen_video,gaze_df,save_dir,win=50,width=1920,height=1080,fps=25):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir,exist_ok=True)
    vidcap = cv2.VideoCapture(screen_video)
    success,image = vidcap.read()
    count = 0
    while success: 
        success,image = vidcap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        try:
            if count%fps==0:
                x = int(list(gaze_df[gaze_df.Frame==count]["gazeX"])[0]*height)
                y = int(list(gaze_df[gaze_df.Frame==count]["gazeY"])[0]*width)
                cropped_image = image[x-win:x+win,y-win:y+win] #,cmap='gray', vmin=0, vmax=255)
                #save_image
                cv2.imwrite(os.path.join(save_dir,str(count)+".jpg"), cropped_image)
        except Exception:
            pass
        count += 1

In [98]:
generate_fucosed_region(screen_video,gaze_df,save_dir="./data/exp1/20220506T080132Z/cropped_imgs")

KeyboardInterrupt: 